In [ ]:
import re
from typing import List, Union

from commonroad.common.util import Interval, AngleInterval

from BatchConversion.BatchConverter import BatchConverter
from OpenSCENARIO2CR.AbsRel import AbsRel
from OpenSCENARIO2CR.ConversionStatistics import ConversionStatistics
from OpenSCENARIO2CR.EsminiWrapper.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter, EFailureReason

In [ ]:
converter = Osc2CrConverter(
    delta_t=0.1,
    goal_state_position_width=5,
    goal_state_position_length=25,
    goal_state_position_use_ego_rotation=True,
    goal_state_time_step=AbsRel[Interval](Interval(-5, 0), is_absolute=False),
    goal_state_orientation=AbsRel[AngleInterval](AngleInterval(-0.8, 0.8), is_absolute=False),

    keep_ego_vehicle=True,
    do_run_cr_monitor=True,
    do_trim_scenario=False,

    ego_filter=re.compile(r".*ego.*", re.IGNORECASE),

    use_implicit_odr_file=True,

    max_time=120.0,
    grace_time=1.0,
    ignored_level=EStoryBoardElementType.ACT,
    log_to_console=False,
    log_to_file=False,
)
batch_converter = BatchConverter(converter)

In [ ]:
from os.path import expanduser

batch_converter.discover_files(expanduser("~"), re.compile(r".*\.xosc", re.IGNORECASE), recursively=True)

In [ ]:
results = batch_converter.run_batch_conversion()

In [ ]:
from commonroad.scenario.scenario import Tag
from commonroad.planning.planning_problem import PlanningProblemSet
from commonroad.common.file_writer import CommonRoadFileWriter
from os import path
import yaml

store_scenarios = True
store_statistics = True
storage_path = "results"

for scenario_path, result in results.items():
    storage_path_without_extension = path.join(storage_path, path.splitext(path.basename(scenario_path))[0])
    if result[0]:
        if isinstance(result[1], EFailureReason):
            print(f"{path.basename(scenario_path)}: {result[1]}")
            continue
        scenario, planning_problem_set, statistics = result[1]
        if store_scenarios and scenario is not None:
            if planning_problem_set is None:
                planning_problem_set = PlanningProblemSet()
            writer = CommonRoadFileWriter(
                scenario=scenario,
                planning_problem_set=planning_problem_set,
                author="Michael Ratzel",
                affiliation="TUM",
                source="Esmini",
                tags={Tag.SIMULATED}
            )
            print(f"{scenario_path} -> {storage_path_without_extension}.xml")
            writer.write_to_file(storage_path_without_extension + ".xml")
        if store_statistics and statistics is not None and isinstance(statistics, ConversionStatistics):
            with open(storage_path_without_extension + ".yaml", mode="w") as file:
                yaml.safe_dump(statistics.to_dict(), file)
    else:
        print(f"{path.basename(scenario_path)}: {result[1]}")


In [ ]:
for scenario_path, result in results.items():
    if not result[0]:
        print("{:40s} {}".format(path.basename(scenario_path), result[1][0]))

In [ ]:
counts = {
    "conversions_total": 0,
    "conversion_successful": 0,
    "conversion_failed": 0,
    "conversion_errored": 0,
    "total_obstacles": 0,
    "failed_obstacles": 0,
    "monitor_success": 0,
    "monitor_success_partially": 0,
    "compliance_checks": 0,
    "compliant_monitor_R_G1": 0,
    "compliant_monitor_R_G2": 0,
    "compliant_monitor_R_G3": 0,
}

for scenario_path, result in results.items():
    counts["conversions_total"] += 1
    if result[0] and not isinstance(result[1], EFailureReason):
        scenario, planning_problem_set, statistics = result[1]
        counts["conversion_successful"] += 1
        counts["total_obstacles"] += len(scenario.dynamic_obstacles)
        counts["failed_obstacles"] += len(statistics.failed_obstacle_conversions)
        if statistics.cr_monitor_analysis is not None:
            any_values_not_none = False
            all_values_not_none = True
            for values in statistics.cr_monitor_analysis.values():
                if values is None:
                    all_values_not_none = False
                else:
                    any_values_not_none = True
                    counts["compliance_checks"] += 1
                    if all([value >= 0.0 for value in values["R_G1"]]):
                        counts["compliant_monitor_R_G1"] += 1
                    if all([value >= 0.0 for value in values["R_G2"]]):
                        counts["compliant_monitor_R_G2"] += 1
                    if all([value >= 0.0 for value in values["R_G3"]]):
                        counts["compliant_monitor_R_G3"] += 1
            if all_values_not_none:
                counts["monitor_success"] += 1
            elif any_values_not_none:
                counts["monitor_success_partially"] += 1

    elif result[0]:
        counts["conversion_failed"] += 1
    else:
        counts["conversion_errored"] += 1


def percentage(description: str, count_part: Union[str, List[str]], count_total: str, invert: bool = False):
    part_sum = counts[count_part] if isinstance(count_part, str) else sum([counts[part] for part in count_part])
    res = 100.0 * part_sum / counts[count_total]
    print("{:30s} {:5.1f} %".format(description + ":", 100 - res if invert else res))


print("{:30s} {:5d}".format("Total num scenarios:", counts["conversions_total"]))
percentage("Conversion success rate", "conversion_successful", "conversions_total")
percentage("Conversion failure rate", "conversion_failed", "conversions_total")
percentage("Conversion error rate", "conversion_errored", "conversions_total")
percentage("Obstacle conversion rate", "failed_obstacles", "total_obstacles", invert=True)
percentage("CR monitor full success rate", "monitor_success", "conversion_successful")
percentage("        partial success rate", "monitor_success_partially", "conversion_successful")
percentage(" full + partial success rate", ["monitor_success", "monitor_success_partially"], "conversion_successful")
percentage("      R_G1 compliance", "compliant_monitor_R_G1", "compliance_checks")
percentage("      R_G2 compliance", "compliant_monitor_R_G2", "compliance_checks")
percentage("      R_G3 compliance", "compliant_monitor_R_G3", "compliance_checks")

In [ ]:
for filepath in results.keys():
    print(filepath)

In [ ]:
for filepath, result in results.items():
    if result[0] is True and isinstance(result[1], EFailureReason):
        print(result[1].name, filepath)
    elif not result[0]:
        continue
        print(filepath, result[1][0])